<a href="https://colab.research.google.com/github/irenebenedetto/machine_learning_iot/blob/main/homework1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 1


## Import the libraries

In [1]:
import datetime
import time
import argparse
import tensorflow as tf
import numpy as np
import pandas as pd
import csv
from os import path

"""
parser=argparse.ArgumentParser()
parser.add_argument('-input',type=float, help='input file folder')
parser.add_argument('-output',type=float, help='period in seconds')

args = parser.parse_args()


input_path = args.input
output_file = args.output
folder = input_path.split('/')[0]

"""

folder = "/content/"

input_path = folder + "input.csv"
output_file = folder + "fusion.tfrecord"


## Data Preparation: Sensor Fusion Dataset with TFRecord

In [2]:
tot_size = path.getsize(input_path)

# open the TFRecordWriter class
with tf.io.TFRecordWriter(output_file) as writer:
  with open(input_path) as input_file:
    lines = csv.reader(input_file)
    for fields in lines:
      
      date = fields[0]
      hour = fields[1]
      temp = int(fields[2])
      humidity = int(fields[3])

      path_audio = folder + fields[4]
      tot_size += path.getsize(path_audio)

      # convert date into posix int 
      dt = datetime.datetime.strptime(date + " " + hour, "%d/%m/%Y %H:%M:%S")
      posix = int(time.mktime(dt.timetuple()))

      # create the features 
      posix_feature = tf.train.Feature(int64_list=tf.train.Int64List(value=[posix])) 
      temp_feature = tf.train.Feature(int64_list=tf.train.Int64List(value=[temp])) 
      humidity_feature = tf.train.Feature(int64_list=tf.train.Int64List(value=[humidity])) 

      # read audio file
      raw_audio = tf.io.read_file(path_audio)
      audio, _ = tf.audio.decode_wav(raw_audio)

      audio_feature = tf.train.Feature(float_list=tf.train.FloatList(value=audio.numpy().flatten().tolist()))

      # crate the dictionary for the structure of each record
      mapping = {
          
          '1': posix_feature, 
          '2': temp_feature,
          '3': humidity_feature,
          '4': audio_feature,

      }
      example = tf.train.Example(features=tf.train.Features(feature=mapping)) 
      writer.write(example.SerializeToString())

red_size = path.getsize(output_file)

print(f"From {str(tot_size/(1024.*1024.))} to {str(red_size/(1024.*1024.))} Mb")



From 0.34406089782714844 to 0.6878013610839844 Mb
